In [1]:
#!pip install pyarrow
#!pip install fastparquet


In [1]:
import pandas as pd
import sqlite3
import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import KernelPCA



In [7]:
bts_faa_sample_1_42 = pd.read_parquet('/home/ccecilia/Downloads/bts_faa_sample_1_42.parquet')

In [2]:
on_time_clean = pd.read_csv('/home/ccecilia/on_time_clean_org.csv')

In [3]:
# select flights that are not cancelled and not diverted
on_time_pca = on_time_clean[(on_time_clean['Cancelled'] == 0) & (on_time_clean['Diverted'] == 0)] #use this line after importing the csv


In [4]:
# convert airline code to numeric
airline_mapping = {
    '9E': 1,
    'AA': 2,
    'AS': 3,
    'B6': 4,
    'DL': 5,
    'EV': 6,
    'F9': 7,
    'G4': 8,
    'HA': 9,
    'MQ': 10,
    'NK': 11,
    'OH': 12,
    'OO': 13,
    'QX': 14,
    'UA': 15,
    'US': 16,
    'VX': 17,
    'WN': 18,
    'YV': 19,
    'YX': 20
}

default_mapping = -1
on_time_pca = on_time_pca.copy()
on_time_pca['airline_int'] = [airline_mapping.get(val, default_mapping) for val in on_time_pca['IATA_CODE_Reporting_Airline']]

In [5]:
TimeBlk_mapping = {
    '0001-0559': 0,
    '0600-0659': 6,
    '0700-0759': 7,
    '0800-0859': 8,
    '0900-0959': 9,
    '1000-1059': 10,
    '1100-1159': 11,
    '1200-1259': 12,
    '1300-1359': 13,
    '1400-1459': 14,
    '1500-1559': 15,
    '1600-1659': 16,
    '1700-1759': 17,
    '1800-1859': 18,
    '1900-1959': 19,
    '2000-2059': 20,
    '2100-2159': 21,
    '2200-2259': 22,
    '2300-2359': 23
}

on_time_pca['DepTimeBlk_int'] = [TimeBlk_mapping.get(val, default_mapping) for val in on_time_pca['DepTimeBlk']]
on_time_pca['ArrTimeBlk_int'] = [TimeBlk_mapping.get(val, default_mapping) for val in on_time_pca['ArrTimeBlk']]

In [6]:
for c in on_time_pca.columns:
    if on_time_pca[c].dtype == 'object':
        print(c)

IATA_CODE_Reporting_Airline
Tail_Number
DepTimeBlk
ArrTimeBlk


In [10]:
on_time_pca.drop(columns=['IATA_CODE_Reporting_Airline','DepTimeBlk','ArrTimeBlk','Tail_Number','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','Cancelled', 'Diverted'], inplace=True)
on_time_pca.reset_index(drop=True, inplace=True)

In [11]:
test = on_time_pca.copy()

In [116]:
'''
def convert_hhmm_to_timestamp(df, col_name):
    
    df[col_name] = pd.to_numeric(df[col_name], errors='coerce', downcast='integer')
    df[col_name] = df[col_name].apply(lambda x: str(int(x)) if str(x).isdigit() else str(x)).str.zfill(4)
    df['Hour'] = df[col_name].str[:2].astype(int)
    if min(df['Hour']) > 0:
        df['Hour'] = df['Hour']-1
    df['Hour'] = df['Hour'].apply(lambda x: str(int(x)) if str(x).isdigit() else str(x)).str.zfill(2)
    df['Year_str'] = df['Year'].astype(str)
    df['Month_str'] = df['Month'].apply(lambda x: str(int(x)) if str(x).isdigit() else str(x)).str.zfill(2)
    df['DayofMonth_str'] = df['DayofMonth'].apply(lambda x: str(int(x)) if str(x).isdigit() else str(x)).str.zfill(2)
    df['ymdhhmm'] = df['Year_str']+df['Month_str']+df['DayofMonth_str']+df['Hour']+df[col_name].str[2:]
    
    
    # Format the result as 'yyyy-mm-dd hh:mm'
    df['ymdhhmm'] = pd.to_datetime(df['ymdhhmm'])
    df[col_name+'_Timestamp'] = df['ymdhhmm'].timestamp()

    # Drop the intermediate columns if needed
    df.drop(columns=[col_name, 'Year_str', 'Month_str', 'DayofMonth_str', 'ymdhhmm', 'Hour'], inplace=True)
    return df
'''

In [12]:
def convert_hhmm_to_timestamp(df, col_name):
    # Convert the 'col_name' column to a numeric (integer) type
    df[col_name] = pd.to_numeric(df[col_name], errors='coerce', downcast='integer')
    
    # Fill missing values with 0
    df[col_name].fillna(0, inplace=True)
    
    # Convert to string and zero-fill to 4 characters
    df[col_name] = df[col_name].astype(int).astype(str).str.zfill(4)
    
    # Extract hours and minutes
    df['Hour'] = df[col_name].str[:2].astype(int)
    df['Minute'] = df[col_name].str[2:]
    
    # Adjust hours
    df['Hour'] = df['Hour'].apply(lambda x: x - 1 if x > 0 else x)
    
    # Convert to string and zero-fill to 2 characters
    df['Hour'] = df['Hour'].astype(int).astype(str).str.zfill(2)
    
    # Convert year, month, and day to strings and zero-fill where necessary
    df['Year_str'] = df['Year'].astype(str)
    df['Month_str'] = df['Month'].astype(int).astype(str).str.zfill(2)
    df['DayofMonth_str'] = df['DayofMonth'].astype(int).astype(str).str.zfill(2)
    
    # Concatenate the components to form 'ymdhhmm' string
    df['ymdhhmm'] = df['Year_str'] + df['Month_str'] + df['DayofMonth_str'] + df['Hour'] + df['Minute']
    
    # Convert 'ymdhhmm' to datetime
    df['ymdhhmm'] = pd.to_datetime(df['ymdhhmm'])
    
    # Convert datetime to timestamp
    df[col_name+'_Timestamp'] = df['ymdhhmm'].astype(int) // 10**9  # Convert nanoseconds to seconds
    
    # Drop intermediate columns
    df.drop(columns=[col_name, 'Year_str', 'Month_str', 'DayofMonth_str', 'ymdhhmm', 'Hour', 'Minute'], inplace=True)
    
    return df

In [14]:
#below code will take about 3 hours
start_time = time.time()

convert_hhmm_to_timestamp(test, 'CRSDepTime')
convert_hhmm_to_timestamp(test, 'DepTime')
convert_hhmm_to_timestamp(test, 'WheelsOff')
convert_hhmm_to_timestamp(test, 'WheelsOn')
convert_hhmm_to_timestamp(test, 'CRSArrTime')
convert_hhmm_to_timestamp(test, 'ArrTime')

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")

Elapsed time: 9261.0847 seconds


In [106]:
test.head(1)

,Year,Quarter,Month,DayofMonth,DayOfWeek,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,DestAirportID,...,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,DivAirportLandings,airline_int,DepTimeBlk_int,ArrTimeBlk_int
0,2015,1,1,1,4,1,12478,1247802,31703,12892,...,390.0,402.0,378.0,1,2475,10,0.0,2,9,12


In [15]:
test.to_csv('test_PCA.csv', index=False)

In [16]:
test.fillna(-1, inplace=True)

In [20]:
test.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Flight_Number_Reporting_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'DestAirportID',
       'DestAirportSeqID', 'DestCityMarketID', 'DepDelay', 'DepDelayMinutes',
       'DepDel15', 'DepartureDelayGroups', 'TaxiOut', 'TaxiIn', 'ArrDelay',
       'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'CRSElapsedTime',
       'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'DistanceGroup',
       'DivAirportLandings', 'airline_int', 'DepTimeBlk_int', 'ArrTimeBlk_int',
       'CRSDepTime_Timestamp', 'DepTime_Timestamp', 'WheelsOff_Timestamp',
       'WheelsOn_Timestamp', 'CRSArrTime_Timestamp', 'ArrTime_Timestamp'],
      dtype='object')

In [21]:
drop_columns = ['CarrierDelay', 
                'WeatherDelay', 
                'NASDelay',
                'SecurityDelay',
                'LateAircraftDelay',
                'Cancelled', 
                'Diverted',
                'DepDelay', 
                'DepDelayMinutes', 
                'DepDel15',
                'ArrDelay', 
                'ArrDelayMinutes', 
                'ArrDel15'
               ]
on_time_clean.drop(columns=drop_columns, inplace=True)
on_time_clean.reset_index(drop=True, inplace=True)

In [25]:
# Standardize the data (important for PCA)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(test)

# Initialize PCA with the number of components you want to keep
n_components = 6  # You can choose the number of components you want to retain
pca = PCA(n_components=n_components)

In [26]:
# Fit PCA to the standardized data
pca.fit(scaled_data)

# Transform the data into the new feature space
transformed_data = pca.transform(scaled_data)

In [27]:
# Calculate the explained variance manually
explained_variance = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance)

print("Explained Variance:", explained_variance)
print("Cumulative Explained Variance:", cumulative_explained_variance)

Explained Variance: [0.1905362  0.17469654 0.13799357 0.06944725 0.06750641 0.05357437]
Cumulative Explained Variance: [0.1905362  0.36523274 0.50322631 0.57267356 0.64017998 0.69375435]


The above results don't seem ideal. :(